First, check we are in the main project directory:

In [1]:
from os import getcwd
print(getcwd())

/Users/u1874623/Documents/GitHub/covid-19-in-households-public/tutorials


If we are instead in the `tutorials` subdirectory, move up to it by uncommenting this line:

In [2]:
cd ..

/Users/u1874623/Documents/GitHub/covid-19-in-households-public


Import functions etc:

In [3]:
from argparse import ArgumentParser
from os import mkdir
from os.path import isdir, isfile
from pickle import load, dump
from copy import deepcopy
from multiprocessing import Pool
from numpy import append, arange, array, exp, log, ones, sum, vstack, where
from numpy.linalg import eig
from numpy.random import rand
from pandas import read_csv
from time import time as get_time
from scipy.integrate import solve_ivp, trapezoid
from matplotlib.pyplot import subplots
from matplotlib.cm import get_cmap
from model.preprocessing import (add_vuln_class, add_vulnerable_hh_members,
estimate_beta_ext, HouseholdPopulation, make_initial_condition_by_eigenvector,
map_SEPIR_to_SEPIRQ,SEPIRInput, SEPIRQInput)
from model.common import SEPIRRateEquations, SEPIRQRateEquations
from model.imports import NoImportModel
from model.specs import (TWO_AGE_UK_SPEC, TWO_AGE_EXT_SEPIRQ_SPEC,
TWO_AGE_SEPIR_SPEC_FOR_FITTING)

Constants and functions we need to run the analysis:

In [4]:
if isdir('outputs/oohi') is False:
    mkdir('outputs/oohi')

DOUBLING_TIME = 10
growth_rate = log(2) / DOUBLING_TIME

SEPIR_SPEC = {**TWO_AGE_SEPIR_SPEC_FOR_FITTING, **TWO_AGE_UK_SPEC}
OOHI_SPEC = {**TWO_AGE_UK_SPEC, **TWO_AGE_EXT_SEPIRQ_SPEC}

relative_iso_rates = [1.0, 1.0, 0.5]

vuln_ext_scale = 0


# List of observed household compositions
composition_list = read_csv(
    'inputs/eng_and_wales_adult_child_vuln_composition_list.csv',
    header=0).to_numpy()
# Proportion of households which are in each composition
comp_dist = read_csv(
    'inputs/eng_and_wales_adult_child_vuln_composition_dist.csv',
    header=0).to_numpy().squeeze()

if isfile('outputs/oohi/beta_ext.pkl') is True:
    with open('outputs/oohi/beta_ext.pkl', 'rb') as f:
        beta_ext = load(f)
else:
    # List of observed household compositions
    fitting_comp_list = read_csv(
        'inputs/eng_and_wales_adult_child_composition_list.csv',
        header=0).to_numpy()
    # Proportion of households which are in each composition
    fitting_comp_dist = read_csv(
        'inputs/eng_and_wales_adult_child_composition_dist.csv',
        header=0).to_numpy().squeeze()
    model_input_to_fit = SEPIRInput(SEPIR_SPEC,
                                    fitting_comp_list,
                                    fitting_comp_dist)
    household_population_to_fit = HouseholdPopulation(
        fitting_comp_list, fitting_comp_dist, model_input_to_fit)
    print('number of states for 2-class pop is',
            household_population_to_fit.Q_int.shape)
    rhs_to_fit = SEPIRRateEquations(model_input_to_fit,
                                    household_population_to_fit,
                                    NoImportModel(5,2))
    beta_ext = estimate_beta_ext(household_population_to_fit,
                                 rhs_to_fit,
                                 growth_rate)
    with open('outputs/oohi/beta_ext.pkl', 'wb') as f:
        dump(beta_ext, f)

vuln_prop = 2.2/56
adult_class = 1

import_model = NoImportModel(6,3)

prev=1.0e-3 # Starting prevalence
starting_immunity=1e-2 # Starting antibody prev/immunity

Class for running the analysis:

In [5]:
pre_npi_input =  SEPIRInput(SEPIR_SPEC,
                                 composition_list,
                                 comp_dist)
pre_npi_input.k_ext *= beta_ext
pre_npi_input = add_vuln_class(pre_npi_input,
                    vuln_prop,
                    adult_class,
                    vuln_ext_scale)
pre_npi_household_population = HouseholdPopulation(
    composition_list, comp_dist, pre_npi_input)
pre_npi_rhs = SEPIRRateEquations(
    pre_npi_input,
    pre_npi_household_population,
    import_model)
H0_pre_npi = make_initial_condition_by_eigenvector(
                                    growth_rate,
                                    pre_npi_input,
                                    pre_npi_household_population,
                                    pre_npi_rhs,
                                    prev,
                                    starting_immunity)


if isfile('outputs/oohi/map_matrix.pkl') is True:
    with open('outputs/oohi/map_matrix.pkl', 'rb') as f:
        map_matrix = load(f)
else:
    print('calculating map matrix...')
    basic_npi_spec = deepcopy(OOHI_SPEC)
    basic_npi_input = SEPIRQInput(basic_npi_spec,
                                composition_list,
                                comp_dist)
    basic_npi_input.k_ext = beta_ext * basic_npi_input.k_ext
    basic_npi_input = add_vuln_class(basic_npi_input,
                        vuln_prop,
                        adult_class,
                        vuln_ext_scale)
    basic_npi_population = HouseholdPopulation(
        composition_list, comp_dist, basic_npi_input)
    map_mat_start = get_time()
    map_matrix = map_SEPIR_to_SEPIRQ(
                                pre_npi_household_population,
                                basic_npi_population)
    print('map matrix calculation took',
            (get_time()-map_mat_start),
            'seconds')
    map_matrix = map_matrix
    with open('outputs/oohi/map_matrix.pkl', 'wb') as f:
                dump(map_matrix, f)

/Users/u1874623/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:407: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


Precondtioner computed in 0.126539945602417s
multiplier calculation took 1.7349040508270264 seconds.


Do the analysis:

In [8]:
iso_rate = 1/5 # Cases identified on average within 5 days of infection
iso_prob = 0 # Turn off isolation by setting adherence probability to zero

p = [iso_rate, iso_prob]

In [10]:
start_time = get_time()

this_spec = deepcopy(OOHI_SPEC)
this_spec['exp_iso_rate'] = p[0] * relative_iso_rates[0] * ones(2,)
this_spec['pro_iso_rate'] = p[0] * relative_iso_rates[1] * ones(2,)
this_spec['inf_iso_rate'] = p[0] * relative_iso_rates[2] * ones(2,)
this_spec['ad_prob'] = p[1]
model_input = SEPIRQInput(this_spec, composition_list, comp_dist)
model_input.k_ext = beta_ext * model_input.k_ext
model_input = add_vuln_class(model_input,
                    vuln_prop,
                    adult_class,
                    vuln_ext_scale)

household_population = HouseholdPopulation(
    composition_list, comp_dist, model_input)
rhs = SEPIRQRateEquations(
    model_input,
    household_population,
    import_model)

H0_iso = H0_pre_npi * map_matrix

print('k_home=',model_input.k_home)
print('k_ext=',model_input.k_ext)

ng_mat = model_input.sus * ((1/this_spec['recovery_rate']) *
 (model_input.k_home) + \
(1/this_spec['symp_onset_rate']) *
(model_input.k_home ) * array([0.5,0.5,0.5]))
print('next gen=',ng_mat)
print('eigs of next gen matrix=',eig(ng_mat))

'''Record the starting state for future reference:'''

S0 = H0_iso.T.dot(household_population.states[:, ::6]) / \
                model_input.ave_hh_by_class
E0 = H0_iso.T.dot(household_population.states[:, 1::6]) / \
                model_input.ave_hh_by_class
P0 = H0_iso.T.dot(household_population.states[:, 2::6]) / \
                model_input.ave_hh_by_class
I0 = H0_iso.T.dot(household_population.states[:, 3::6]) / \
                model_input.ave_hh_by_class
R0 = H0_iso.T.dot(household_population.states[:, 4::6]) / \
                model_input.ave_hh_by_class
Q0 = H0_iso.T.dot(household_population.states[:, 5::6]) / \
                model_input.ave_hh_by_class


no_days = 100
tspan = (0.0, no_days)
solution = solve_ivp(rhs,
                     tspan,
                     H0_iso,
                     first_step=0.001,
                     atol=1e-16)

t = solution.t
H = solution.y

S = H.T.dot(household_population.states[:, ::6])
E = H.T.dot(household_population.states[:, 1::6])
P = H.T.dot(household_population.states[:, 2::6])
I = H.T.dot(household_population.states[:, 3::6])
R = H.T.dot(household_population.states[:, 4::6])
Q = H.T.dot(household_population.states[:, 5::6])

I_vuln_present = \
    H[
        where(household_population.composition_by_state[:,2]>0)[0],
        :].T.dot(
        household_population.states[
            where(household_population.composition_by_state[:,2]>0)[0],
            3::6]
        )

print('I(at least one vuln present)=',I_vuln_present/model_input.ave_hh_by_class)

vuln_peak = 100 * max(I[:, 2]) / \
                model_input.ave_hh_by_class[2]
vuln_end = 100 * R[-1, 2] / \
                model_input.ave_hh_by_class[2]
iso_peak = 100 * max(Q.sum(axis=1)) / model_input.ave_hh_size
Q_all = Q.sum(axis=1) / model_input.ave_hh_size
cum_iso = 100 * trapezoid(Q_all, t)

print('Calculations took', (get_time()-start_time), 'seconds.')

k_home= [[0.10724038 0.09110216 0.09110216]
 [0.05223182 0.06874136 0.06874136]
 [0.05223182 0.06874136 0.06874136]]
k_ext= [[0.16642145 0.08582382 0.00350952]
 [0.02948755 0.21046348 0.00860631]
 [0.         0.         0.        ]]
next gen= [[0.5898221  0.50106189 0.50106189]
 [0.287275   0.3780775  0.3780775 ]
 [0.287275   0.3780775  0.3780775 ]]
eigs of next gen matrix= (array([ 1.21594506e+00,  1.30032036e-01, -2.63504091e-32]), array([[ 7.49375903e-01,  8.38879070e-01,  3.51833581e-16],
       [ 4.68207089e-01, -3.84890833e-01, -7.07106781e-01],
       [ 4.68207089e-01, -3.84890833e-01,  7.07106781e-01]]))
I(at least one vuln present)= [[8.80269563e-17 1.36423438e-17 1.72354856e-16]
 [3.23177118e-12 9.74769858e-13 1.72326131e-16]
 [3.90014184e-10 1.17637985e-10 1.72039017e-16]
 [3.17035754e-08 9.56256095e-09 1.69484778e-16]
 [1.64104892e-07 4.94973499e-08 1.65750330e-16]
 [5.30488439e-07 1.59999893e-07 1.60283600e-16]
 [1.43341589e-06 4.32284531e-07 1.52022813e-16]
 [3.27438558e-